<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [15]:
import sys
from pathlib import Path

# Ajouter le dossier src au path
src_path = Path("..") / ".."
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2


import os
import uuid
import json
import mlflow
import tempfile
from datetime import datetime
from pathlib import Path
from huggingface_hub import HfApi
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.utils import safe_mean
from dotenv import load_dotenv
import textwrap
from huggingface_hub import hf_hub_download
from env.snake_env import SnakeEnv

load_dotenv()
hf_token = os.getenv("HF_HUB_TOKEN")
if not hf_token:
    raise ValueError("⚠️ Variable HF_HUB_TOKEN manquante.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3 style="color:#CCC229">TRAIN FUNCTION<h3>

In [38]:
train_snake(timesteps=100000, grid_size=5, n_envs=8, algorithm="PPO")

In [59]:
train_snake(timesteps=100000, grid_size=5, n_envs=8, algorithm="PPO", base_uuid="20a0ecbd-360b-401c-9977-a6bfaec87e23")


🔍 Scan du dépôt pour trouver l'UUID : 20a0ecbd-360b-401c-9977-a6bfaec87e23 ...
📍 Fichier trouvé : 5x5/20a0ecbd-360b-401c-9977-a6bfaec87e23/metadata.json
📥 Téléchargement du modèle : 5x5/20a0ecbd-360b-401c-9977-a6bfaec87e23/model.zip
✅ Succès ! Agent chargé (Grille 5x5)
✅ Modèle chargé avec succès. Reprise sur grille 5x5.

🚀 Démarrage Run MLflow : FINE-TUNING_20260125_213701_b3293178
🆔 Nouvel ID : b3293178-4918-4855-ab65-4d2980e883ac
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.28     |
|    ep_rew_mean     | 0.109    |
| time/              |          |
|    fps             | 7345     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 131072   |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 6.54       |
|    ep_rew_mean          | 0.328      |
| time/                   |            |
|    fps          

Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (1 / 1): 100%|██████████|  175kB /  175kB,  292kB/s  
Processing Files (1 / 1): 100%|██████████|  175kB /  175kB,  175kB/s  
New Data Upload: 100%|██████████|  175kB /  175kB,  175kB/s  
2026/01/25 21:38:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/25 21:38:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp8jaf9ckt/model, flavor: python_function). Fall back to return ['cloudpickle==3.1.2']. Set logging level to DEBUG to see the full traceback. 
Registered model 'Snake_5x5' already exists. Creating a new version of this model...
Created version '2' of model 'Snake_5x5'.


✅ Terminé ! Nouveau modèle : b3293178-4918-4855-ab65-4d2980e883ac


(<stable_baselines3.ppo.ppo.PPO at 0x728844bd8e20>,
 'b3293178-4918-4855-ab65-4d2980e883ac')

In [56]:
list_snake_models()

🔍 Recherche des modèles dans Lategardener/snake-rl-models...

             MODELE (UUID)                   ||                 DESCRIPTION
--------------------------------------------------------------------------------------------------------------
05x05 / 20a0ecbd-360b-401c-9977-a6bfaec87e23 || Algo: PPO | Date: 25/01/2026 20:58:59 | Reward: -0.06
05x05 / 6ee34c5d-af9e-48a6-b64f-1b1cc8a0080a || Algo: PPO | Date: 25/01/2026 20:41:04 | Reward: 0.01
--------------------------------------------------------------------------------------------------------------
✅ Nombre de modèles trouvés : 2
